<a href="https://colab.research.google.com/github/feliciakiani/PolitikPedia/blob/main/Machine%20Learning/semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Menggunakan USE

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load the Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Load your dataset
df = pd.read_csv('dataset_semantic.csv')

# Encode questions using Universal Sentence Encoder
question_embeddings = embed(df['Pertanyaan'])

# Define your query
query = "Bagaimana sistem pemilu di Indonesia?,Partai Politik"

# Encode the query
query_embedding = embed([query])[0]

# Calculate cosine similarity between the query and dataset
cosine_similarities = cosine_similarity([query_embedding], question_embeddings)[0]

# Combine similarities with questions
results = list(zip(cosine_similarities, df['Pertanyaan'], df['Pencarian']))

# Sort the results by similarity in descending order
results.sort(reverse=True, key=lambda x: x[0])

# Print the results
for similarity, question, category in results:
    print(f"{similarity:.2f}: {question} ({category})")


In [ ]:
# Print the highest similarity score and corresponding sentence
highest_score, highest_question, highest_category = results[0]
print(f"\nHighest Similarity Score: {highest_score:.2f}")
print(f"Highest Similarity Question: {highest_question} ({highest_category})")

NameError: ignored

In [6]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Lambda, Dropout, Conv1D
from sklearn.metrics.pairwise import cosine_similarity

# Load your dataset
df = pd.read_csv('dataset_semantic.csv')

# Define your dataset
questions = df['Pertanyaan'].values
categories = df['Pencarian'].values

# Find the maximum length of questions
max_length = max(len(question.split()) for question in questions)
print("Maximum Length of Questions:", max_length)

# Tokenize the questions
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions)
sequences = tokenizer.texts_to_sequences(questions)

# Pad sequences for consistent input length
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences)

Maximum Length of Questions: 8


In [7]:
# Create the model using Sequential API
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=padded_sequences.shape[1]),
    Dropout(0.2),
    Conv1D(32, 5, activation='relu'),
    GlobalAveragePooling1D(),
    Dense(50, activation='relu'),
    Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
])

# Compile the model with default cosine similarity metric (you can customize the optimizer and loss)
model.compile(optimizer='adam', loss='mse', metrics=['cosine_similarity'])

# Train the model
target_data = model.predict(padded_sequences)  # Use the same embeddings for target data
model.fit(padded_sequences, target_data, epochs=10, batch_size=32)

313/313 [==============================] - 3s 2ms/step
Epoch 1/10
313/313 [==============================] - 6s 6ms/step - loss: 9.7934e-05 - cosine_similarity: 0.9976
Epoch 2/10
313/313 [==============================] - 2s 5ms/step - loss: 2.1747e-05 - cosine_similarity: 0.9995
Epoch 3/10
313/313 [==============================] - 1s 4ms/step - loss: 1.3121e-05 - cosine_similarity: 0.9997
Epoch 4/10
313/313 [==============================] - 1s 4ms/step - loss: 9.1465e-06 - cosine_similarity: 0.9998
Epoch 5/10
313/313 [==============================] - 1s 4ms/step - loss: 7.1032e-06 - cosine_similarity: 0.9998
Epoch 6/10
313/313 [==============================] - 1s 4ms/step - loss: 5.4851e-06 - cosine_similarity: 0.9999
Epoch 7/10
313/313 [==============================] - 1s 5ms/step - loss: 4.2779e-06 - cosine_similarity: 0.9999
Epoch 8/10
313/313 [==============================] - 2s 6ms/step - loss: 3.6789e-06 - cosine_similarity: 0.9999
Epoch 9/10
313/313 [=====================

In [8]:
# Encode the query
query = "p"
query_sequence = tokenizer.texts_to_sequences([query])
query_padded = tf.keras.preprocessing.sequence.pad_sequences(query_sequence, maxlen=padded_sequences.shape[1])

# Get the embeddings for the query
query_embedding = model.predict(query_padded)

# Calculate cosine similarity between the query embedding and dataset embeddings
cosine_similarities = cosine_similarity(query_embedding, model.predict(padded_sequences))

# Combine similarities with questions
results = list(zip(cosine_similarities[0], questions, categories))

# Sort the results by similarity in descending order
results.sort(reverse=True, key=lambda x: x[0])

# Print the results
for similarity, question, category in results:
    print(f"{similarity:.2f}: {question} ({category})")


Streaming output truncated to the last 5000 lines.
0.79: Apa peran pemuda dalam partai politik? (Jenis Partai Politik)
0.79: Apa peran pemuda dalam partai politik? (Peran Pemuda Politik)
0.79: Apa peran pemuda dalam partai politik? (Jenis Partai Politik)
0.79: Apa peran pemuda dalam partai politik? (Pengaruh Partai Politik)
0.79: Apa peran pemuda dalam partai politik? (Pengaruh Partai Politik)
0.79: Apa peran pemuda dalam partai politik? (Jenis Partai Politik)
0.79: Apa peran pemuda dalam partai politik? (Pengaruh Partai Politik)
0.79: Apa peran pemuda dalam partai politik? (Politik Koalisi)
0.79: Apa peran pemuda dalam partai politik? (Peran Pemuda Politik)
0.79: Apa peran pemuda dalam partai politik? (Jenis Partai Politik)
0.79: Apa peran pemuda dalam partai politik? (Pengaruh Partai Politik)
0.79: Apa peran pemuda dalam partai politik? (Proses Pemilu)
0.79: Apa peran pemuda dalam partai politik? (Pengaruh Partai Politik)
0.79: Apa peran pemuda dalam partai politik? (Proses Pemilu)
0

In [9]:
# Print the highest similarity score and corresponding sentence
highest_score, highest_question, highest_category = results[0]
print(f"\nHighest Similarity Score: {highest_score:.2f}")
print(f"Highest Similarity Question: {highest_question} ({highest_category})")


Highest Similarity Score: 0.98
Highest Similarity Question: Bagaimana partai politik mempengaruhi pemerintahan? (Jenis Partai Politik)


In [ ]:
import pickle

# Save the entire model to an H5 file
model.save("semantic_search_model.h5")

# Save only the weights of the model to a file
model.save_weights("semantic_search_model_weights.h5")

# Save the tokenizer to a file
tokenizer_path = "tokenizer.pickle"
with open(tokenizer_path, 'wb') as handle:
    pickle.dump(tokenizer, handle)

# Load the entire model from the H5 file
model_path = "semantic_search_model.h5"
model = tf.keras.models.load_model(model_path)

# Save the model and tokenizer in SavedModel format
export_path = "saved_model/1"
model.save(export_path, save_format="tf")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from google.colab import files

# Save the best model checkpoint to a file
files.download('/content/semantic_search_model')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the weights into the model
weights_path = "/content/semantic_search_model_weights.h5"
model.load_weights(weights_path)

# Encode the query
query = "Apa itu politik"
query_sequence = tokenizer.texts_to_sequences([query])
query_padded = tf.keras.preprocessing.sequence.pad_sequences(query_sequence, maxlen=padded_sequences.shape[1])

# Get the embeddings for the query
query_embedding = model.predict(query_padded)

# Calculate cosine similarity between the query embedding and dataset embeddings
cosine_similarities = cosine_similarity(query_embedding, model.predict(padded_sequences))

# Combine similarities with questions
results = list(zip(cosine_similarities[0], questions, categories))

# Sort the results by similarity in descending order
results.sort(reverse=True, key=lambda x: x[0])

# Print the results
for similarity, question, category in results:
    print(f"{similarity:.2f}: {question} ({category})")

In [ ]:
# Save the questions to a text file
with open('/content/questions.txt', 'w') as file:
    for question in questions:
        file.write(f"{question}\n")

# Download the text file
from google.colab import files
files.download('/content/questions.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/my_directory.zip /content/saved_model
from google.colab import files

files.download("/content/my_directory.zip")


  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/1/ (stored 0%)
  adding: content/saved_model/1/fingerprint.pb (stored 0%)
  adding: content/saved_model/1/keras_metadata.pb (deflated 83%)
  adding: content/saved_model/1/saved_model.pb (deflated 86%)
  adding: content/saved_model/1/variables/ (stored 0%)
  adding: content/saved_model/1/variables/variables.index (deflated 54%)
  adding: content/saved_model/1/variables/variables.data-00000-of-00001 (deflated 20%)
  adding: content/saved_model/1/assets/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>